In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [4]:
df = pd.read_csv('slash.txt',sep='\t')


In [5]:
G = nx.DiGraph()
for index,rows in df.iterrows():
    u = rows['FromNodeId']
    v = rows['ToNodeId']
    s = rows['Sign']
    G.add_edge(u,v,sign=s)

In [6]:
in_pos = {}
in_neg = {}
for nodes in G.nodes():
    l = list(G.in_edges(nodes,data=True))
    pos = 0
    neg = 0
    for items in l:
        if items[2]['sign'] == 1:
            pos += 1
        else:
            neg += 1
    in_pos[nodes] = pos
    in_neg[nodes] = neg

In [7]:
nx.set_node_attributes(G,in_pos,'in_pos')
nx.set_node_attributes(G,in_neg,'in_neg')


In [8]:
out_pos = {}
out_neg = {}
for nodes in G.nodes():
    pos = 0
    neg = 0
    l = dict(G[nodes])
    for keys in l:
        if(l[keys]['sign'] == 1):
            pos += 1
        else:
            neg += 1
    out_pos[nodes] = pos
    out_neg[nodes] = neg

{'in_pos': 199, 'in_neg': 29}

In [11]:
nx.set_node_attributes(G,out_pos,'out_pos')
nx.set_node_attributes(G,out_neg,'out_neg')


{'in_pos': 199, 'in_neg': 29, 'out_pos': 24, 'out_neg': 5}

In [88]:
status_heuristics = {}
for node in Nodes:
    status_heuristics[node[0]] = node[1]["in_pos"] + node[1]["out_neg"]- node[1]["in_neg"] - node[1]["out_pos"]
    
nx.set_node_attributes(G, status_heuristics, 'status_heuristics')
Nodes = G.nodes(data = True)
Nodes[4]['status_heuristics']

93

In [89]:
embed = {}
for edge in list(G.edges()):
    inward_i=[]
    inward_j=[]
    outward_i=[]
    outward_j=[]
    for items in list(G.in_edges(edge[0])):
        inward_i.append(items[0])
    for items in list(G.in_edges(edge[1])):
        inward_j.append(items[0])
    for keys in dict(G[edge[0]]):
        outward_i.append(keys)
    for keys in dict(G[edge[1]]):
        outward_j.append(keys)
    final_i = list(set(inward_i) | set(inward_j))
    final_j = list(set(outward_i) | set(outward_j))
    final = list(set(final_i) & set(final_j)) 
    embed[edge] = len(final)

KeyboardInterrupt: 

In [43]:
nx.set_edge_attributes(G,embed,"embed")

In [106]:
pred_sign = {} 
count = 0;
for edge in list(G.edges()):
    u = edge[0];
    v = edge[1];
    
    x = Nodes[u]['status_heuristics']
    y = Nodes[v]['status_heuristics']
    if x<=y:
        pred_sign[edge] = 1
        count+=1;
        
    else:
        pred_sign[edge] = -1
        


In [107]:
nx.set_edge_attributes(G, pred_sign, 'status_sign')

In [108]:
x = []
x_pred = []
for edge in G.edges(data=True):
    x.append(edge[2]['sign'])
    x_pred.append(edge[2]['status_sign'])

In [109]:
stat_diff = nx.get_edge_attributes(G,'status_sign')
sign = nx.get_edge_attributes(G,'sign')
embed = nx.get_edge_attributes(G,'embed')

df = pd.DataFrame(columns=['edge', "stat_diff", 'sign', 'embed'])

In [110]:
df['edge'] = embed.keys()
df['stat_diff'] = stat_diff.values()
df['sign'] = sign.values()
df['embed'] = embed.values()

In [111]:
y_pred = df["stat_diff"]












y = df["sign"]

In [113]:
from sklearn.metrics import accuracy_score
accuracy_score( y,y_pred)

0.7642397514939858

In [124]:
df1 = df.loc[df['embed']>25]
y_pred = df1["stat_diff"]
y = df1["sign"]

accuracy_score( y,y_pred)

0.6843890144916793